In [4]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset
        
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size , Generate_Naive_Feedforward_Contig, Generate_Naive_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 64

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")


Data loaded from file
Training Samples:  90432456 | Validation Samples:  10048051
Training and Validation datasets separated
Model Compiled


In [5]:
train_gen = Generate_Naive_Feedforward(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                (5,), #label shape
                                input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                normalized = True
                                )

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                (5,), #label shape
                                input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                normalized = True
                                )

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_SGD_128_128v1_normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=20,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

Epoch 1/20
 243/1000 [======>.......................] - ETA: 29:00 - loss: 1.5076 - acc: 0.3319

KeyboardInterrupt: 

In [1]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

def Generate_Naive_Feedforward(dataset, batch_size, feature_shape, feature_transformation_function, label_transformation_function):
    
    features = np.zeros((batch_size, *feature_shape))
    labels = np.zeros((batch_size, 5))
    index_set = set(range(len(dataset)))
    
    while True:
            
        batch_indicies = []    
        if (len(index_set) < batch_size):
            index_set = set(range(len(dataset)))
            batch_indicies = random.sample(index_set, batch_size)
        else:
            batch_indicies = random.sample(index_set, batch_size)
            index_set = index_set.difference(batch_indicies)
        
        for i, batch_index in enumerate(batch_indicies):
            features[i] = feature_transformation_function(dataset[batch_index])
            labels[i] = label_transformation_function(dataset[batch_index])
            
        yield (features, labels)
        
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
from keras.models import load_model
from generator_helper_functions import input_layer, naive_input_size

batch_size = 64

model = load_model('models/naive_feedforward_SGD_128_128v1.hdf5')

print("Loaded")

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: nf_label_transformation_function(x[2]))

model.evaluate_generator(validation_gen,
                    steps=1000,
                    )

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Training Samples:  90432456 | Validation Samples:  10048051
Training and Validation datasets separated


Using TensorFlow backend.


Loaded


[1.378786684513092, 0.3794375]

## Trying different neurons per hidden layer and regression

In [1]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

def Generate_Naive_Feedforward(dataset, batch_size, feature_shape, feature_transformation_function, label_transformation_function, normalized = False):
    
    features = np.zeros((batch_size, *feature_shape))
    labels = np.zeros((batch_size,))
    index_set = set(range(len(dataset)))
    
    while True:
            
        batch_indicies = []    
        if (len(index_set) < batch_size):
            index_set = set(range(len(dataset)))
            batch_indicies = random.sample(index_set, batch_size)
        else:
            batch_indicies = random.sample(index_set, batch_size)
            index_set = index_set.difference(batch_indicies)
        
        for i, batch_index in enumerate(batch_indicies):
            features[i] = feature_transformation_function(dataset[batch_index], normalized)
            labels[i] = label_transformation_function(dataset[batch_index])
            
        yield (features, labels)

import warnings
warnings.simplefilter('ignore')

import keras
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Lambda
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 64

def round(inp):
    return tf.round(inp * 4) / 4

model = Sequential()
model.add(Dense(90, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(90, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.add(Lambda(round))

model.compile(loss='binary_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")

train_gen = Generate_Naive_Feedforward(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1)/4,
                                normalized=True)

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1 )/4,
                                normalized=True)

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_regression_90_90_normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=1,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Training Samples:  90432456 | Validation Samples:  10048051
Training and Validation datasets separated


Using TensorFlow backend.


Model Compiled


ValueError: An operation has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [22]:
model.predict(input_layer(training_data[0]))

ValueError: Error when checking : expected dense_7_input to have shape (497965,) but got array with shape (1,)

In [19]:
input_layer(training_data[0])

array([0.        , 0.        , 0.        , ..., 3.75056865, 4.        ,
       1.00479931])

In [10]:
model.predict()

TypeError: predict() missing 1 required positional argument: 'x'

## Trying different neurons per hidden layer and poisson regression

In [1]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

def Generate_Naive_Feedforward(dataset, batch_size, feature_shape, feature_transformation_function, label_transformation_function):
    
    features = np.zeros((batch_size, *feature_shape))
    labels = np.zeros((batch_size,))
    index_set = set(range(len(dataset)))
    
    while True:
            
        batch_indicies = []    
        if (len(index_set) < batch_size):
            index_set = set(range(len(dataset)))
            batch_indicies = random.sample(index_set, batch_size)
        else:
            batch_indicies = random.sample(index_set, batch_size)
            index_set = index_set.difference(batch_indicies)
        
        for i, batch_index in enumerate(batch_indicies):
            features[i] = feature_transformation_function(dataset[batch_index])
            labels[i] = label_transformation_function(dataset[batch_index])
            
        yield (features, labels)

import warnings
warnings.simplefilter('ignore')

import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 128

model = Sequential()
model.add(Dense(90, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(90, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='poisson',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")

train_gen = Generate_Naive_Feedforward(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1)/4)

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1 )/4)

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_regression_poisson_90_90.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=1,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=10,
                    callbacks=[checkpointer]
                    )

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Training Samples:  90432456 | Validation Samples:  10048051
Training and Validation datasets separated


Using TensorFlow backend.


Model Compiled
Epoch 1/1
 113/1000 [==>...........................] - ETA: 35:28 - loss: 12.3481 - acc: 0.0442

KeyboardInterrupt: 

In [ ]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

import warnings
warnings.simplefilter('ignore')

import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size, Generate_Naive_Feedforward_Contig

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 128

model = Sequential()
model.add(Dense(90, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(90, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='poisson',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")

train_gen = Generate_Naive_Feedforward_Contig(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1)/4)

validation_gen = Generate_Naive_Feedforward_Contig(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1 )/4)

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_regression_poisson_90_90.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=1,
                    steps_per_epoch=500,
                    validation_data=validation_gen,
                    validation_steps=10,
                    callbacks=[checkpointer]
                    )

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Training Samples:  90432456 | Validation Samples:  10048051
Training and Validation datasets separated


Using TensorFlow backend.


Model Compiled
Epoch 1/1
 65/500 [==>...........................] - ETA: 34:16 - loss: 10.5465 - acc: 0.0430